In [ ]:
!pip install langchain
!pip install huggingface-hub
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pypdf

In [ ]:
!pip install scikit-learn
!pip install -U sentence-transformers
!pip install pymupdf


In [ ]:
!pip install pytesseract

In [ ]:
!pip install scikit-learn

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.llms import Ollama
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:
import os
import fitz
import pytesseract
import io
from PIL import Image
import time

# Start the timer
start_time = time.time()

os.environ["PATH"] += os.pathsep + '/opt/homebrew/bin'
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Function to apply OCR to an image
def ocr_image(image):
    img = Image.open(io.BytesIO(image))
    custom_config = r'--oem 3 --psm 6'
    return pytesseract.image_to_string(img, config=custom_config).lower()

# Function to load documents and apply OCR to images within them
def load_data(folder_path):
    data = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            with fitz.open(file_path) as doc:
                for page_num, page in enumerate(doc):
                    # Extract text
                    page_text = page.get_text().lower()

                    # Extract images and apply OCR
                    for img_index, img in enumerate(page.get_images(full=True)):
                        xref = img[0]
                        base_image = doc.extract_image(xref)
                        image_data = base_image["image"]
                        page_text += "\n" + ocr_image(image_data)

                    page_metadata = {'source': file_path, 'page': page_num}
                    data.append(Document(page_text, page_metadata))
    return data

In [ ]:
documents = load_data(folder_path="/Users/meheksawhney/Desktop/UI/attached_documents")
print(len(documents))

In [ ]:
def creating_chunks(documents):
    chunk_list = []
    for doc in documents:
        chunk_list.append(doc)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=30)
    chunks = text_splitter.split_documents(chunk_list)

    return chunks

# class DocumentChunk:
#     def __init__(self, page_content, metadata):
#         self.page_content = page_content
#         self.metadata = metadata

# def creating_chunks(documents):
#     chunk_list = []
#     for doc in documents:
#         if hasattr(doc, 'page_content') and isinstance(doc.page_content, str):
#             sections = doc.page_content.split('\n\n')
#             for section in sections:
#                 # Create a new DocumentChunk for each section
#                 chunk_list.append(DocumentChunk(page_content=section, metadata=doc.metadata))
#         else:
#             print("Document missing page content or 'page_content' is not a string.")
#     return chunk_list


In [ ]:
chuncked_documents = creating_chunks(documents)
print(chuncked_documents)
print(len(chuncked_documents))

In [ ]:
def rank_documents_with_tfidf(query, documents):
    tfidf_vectorizer = TfidfVectorizer()
    
    # Extract text content from each document for TF-IDF vectorization
    document_texts = [doc.page_content for doc in documents]
    tfidf_vectors = tfidf_vectorizer.fit_transform(document_texts)

    query_vector = tfidf_vectorizer.transform([query])
    cosine_similarities = linear_kernel(query_vector, tfidf_vectors).flatten()

    # Rank all documents based on their cosine similarity
    ranked_document_indices = cosine_similarities.argsort()[::-1]
    ranked_documents = [(documents[i], cosine_similarities[i]) for i in ranked_document_indices]
    return ranked_documents

# def rank_documents_with_tfidf(query, documents):
#     tfidf_vectorizer = TfidfVectorizer()
#     document_texts = [doc.page_content for doc in documents]
#     tfidf_vectors = tfidf_vectorizer.fit_transform(document_texts)
#     query_vector = tfidf_vectorizer.transform([query])
#     cosine_similarities = linear_kernel(query_vector, tfidf_vectors).flatten()

#     ranked_document_indices = cosine_similarities.argsort()[::-1]
#     ranked_documents = [(documents[i], cosine_similarities[i]) for i in ranked_document_indices]


#     # # Log the top retrieved documents to check their relevance
#     # for doc, score in ranked_documents[:5]:  # Adjust the number as needed
#     #     print(f"Doc: {doc.metadata['source']}, Score: {score}, Text Preview: {doc.page_content[:150]}")  # Preview first 150 characters

#     return ranked_documents

In [ ]:
# Loading llama2 from Ollama
!ollama pull llama2:13b
llama_model = Ollama(model="llama2:13b")

In [ ]:
def embedding_model_LLAMA_2(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # running the model on CPU
        encode_kwargs={'normalize_embeddings': normalize_embedding} # ensure this is true
    )
# Loading the Embedding Model for LLAMA-2
embeddingModel = embedding_model_LLAMA_2(model_path="all-MiniLM-L6-v2")

In [ ]:
vectorstore_LLAMA_2 = FAISS.from_documents(chuncked_documents, embeddingModel)

# Creating the chain for Question Answering
qa_chain_llama2 = RetrievalQA.from_chain_type(
        llm=llama_model,
        retriever=vectorstore_LLAMA_2.as_retriever(),
        chain_type="stuff",
        return_source_documents=True, #displays where the answer is coming from
    )

In [ ]:
!pip install rouge
!pip install nltk

import nltk
nltk.download('punkt')


In [ ]:

from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize

def query(query_text, llama_model, documents, embeddingModel):
    # Get ranked documents with their cosine scores
    query_text = query_text.lower()
    tfIdf_documents = rank_documents_with_tfidf(query_text, documents)

    # Extract just the relevant documents from the tfIdf documents
    relevant_documents = [doc for doc, _ in tfIdf_documents]

    # Process only preselected documents
    relevant_chunks = creating_chunks(relevant_documents)

    vectorstore_LLAMA2 = FAISS.from_documents(relevant_chunks, embeddingModel)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llama_model,
        retriever=vectorstore_LLAMA2.as_retriever(),
        chain_type="stuff",
        return_source_documents=True
    )

    # Send the query to the temp qa_chain
    response = qa_chain({'query': query_text})

    cosine_scores = []
    for doc, score in tfIdf_documents[:4]:
        cosine_scores.append(score)


    content = []
    pages = []
    document_name = []

    # Process 'source_documents'
    if 'source_documents' in response:
        for doc in response['source_documents']:
            if hasattr(doc, 'metadata') and 'page' in doc.metadata:
                pages.append(doc.metadata['page'] + 1)
            if hasattr(doc, 'page_content'):
                content.append(doc.page_content)
                # reference_texts.append(doc.page_content)  # Add to reference texts
            if hasattr(doc, 'metadata') and 'source' in doc.metadata:
                document_name.append(doc.metadata['source'])

        # Remove 'source_documents' from the response for the final output
        del response['source_documents']
        
        return {
            "response": response,
            "document_names": list(set(document_name)),
            "cosine_scores": cosine_scores,
            "content": content,
            "pages": pages if pages else "No page available",
        }

query("Can you provide a summarisation for the article exploring open information extraction for Portuguese using Large Language Models?", llama_model, documents, embeddingModel)
# end_time = time.time()

# elapsed_time = end_time - start_time
# print(f"The elapsed time is {elapsed_time} seconds.")

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Sample texts
# generated_summary = "sure! Here is a summary of the article 'Exploring Open Information Extraction for Portuguese Using Large Language Models':\n\nThe article discusses the use of large language models (LLMs) for open information extraction (OIE) in the portuguese language. The authors explore the applicability of LLMs for OIE in portuguese, examining various approaches and techniques such as few-shot and prompt engineering. They also compare the performance of an llm with a state-of-the-art oie system for portuguese and discuss future research directions.\n\nThe article is structured as follows:\n\n* Section 2 reviews related work on OIE and LLMs.\n* Section 3 outlines the methodology and approach employed in the study.\n* Section 4 presents the results of experiments conducted to evaluate the performance of the llm for OIE in portuguese.\n* Section 5 concludes with a discussion of the findings and future research directions.\n\nOverall, the article provides an exploratory analysis of LLMs for OIE in the portuguese language and highlights their potential for improving the accuracy and efficiency of OIE systems in this language."
# expert_summary = "In this work, we investigate the potential of Large Language Models (LLMs) for Open Information Extraction (OpenIE) in the Por- tuguese language. While most OpenIE meth- ods are primarily optimized for English, only few works in the literature explore their uses for cross-lingual and multilingual scenarios. De- spite the growing interest in Portuguese OpenIE methods, the use LLMs for Portuguese focused OpenIE is still an underdeveloped topic in the area.Our study addresses this research gap by examining the viability of using open and com- mercial LLMs with few-shot prompt engineer- ing for Portuguese OpenIE. We provide an anal- ysis of the performance of these LLMs in Ope- nIE tasks, revealing that they achieve perfor- mance metrics comparable to state-of-the-art systems. In addition, we have fine-tuned and launched an open LLM for OpenIE (PortOIE- Llama), which outperforms commercial LLMs in our experiments. Our findings highlight the potential of LLMs in Portuguese OpenIE tasks and suggest that further refinement and fine- tuning of larger models could enhance these results."

# # Create a vectorizer object
# vectorizer = TfidfVectorizer()

# # Transform texts to TF-IDF vectors
# tfidf_matrix = vectorizer.fit_transform([generated_summary, expert_summary])

# # Calculate cosine similarity (the output is a matrix where each row and column represent texts, and each value is the similarity score)
# similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# print("Cosine similarity between the generated summary and expert summary:")
# print(similarity_score[0][0])




In [ ]:
def query_check(query_text, llama_model, documents, embeddingModel):
    # Lowercase the query text for consistent matching
    query_text = query_text.lower()

    # Get ranked documents with their cosine scores
    ranked_documents = rank_documents_with_tfidf(query_text, documents)

    # Select top 10 documents considered as most relevant based on their cosine similarity
    top_relevant_documents = [doc for doc, _ in ranked_documents[:10]]

    # Continue with processing only the selected documents
    relevant_chunks = creating_chunks(top_relevant_documents)
    vectorstore_LLAMA_2 = FAISS.from_documents(relevant_chunks, embeddingModel)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llama_model,
        retriever=vectorstore_LLAMA_2.as_retriever(),
        chain_type="stuff",
        return_source_documents=True
    )
    response = qa_chain({'query': query_text})

    # Collect results
    document_names = [doc.metadata['source'] for doc in top_relevant_documents]
    pages = [doc.metadata['page'] for doc in top_relevant_documents]

    return response, document_names, pages

In [ ]:
gold_standard = [
{
  "query": "How many years do Major reviews take?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/quality-assurance-code-of-practice-approach-to-code-development.pdf"
  ],
  "expected_pages": [2],
  "expected_answers": ["1-2 years"]
},
{
  "query": "Who supports the doctoral students throughout their resgisteration?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["The supervisory teams are responsible for ensuring the doctoral students receive appropriate guidance and support throughout their registeration with the University."]
},
{
  "query": "What is CPD defined as?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [1],
  "expected_answers": ["A range of short and long training programmes for learners...who are undertaking the course for purposes of professional development/upskilling/workforce development"]
},
{
  "query": "Who is responsible for signing off the final key documentation for the new course?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa3-approval-of-new-courses-of-study.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["The Chair of the F/SL TQC"]
},
{
  "query": "Who monitors the doctoral student progression?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["The Associate Deans for Research"]
},
{
  "query": "What is the dealine to approve Phase 2 and 3 proposal",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa-ct-phase_2-3-approvals-2023-24.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["March to May 2024"]
},
{
  "query": "What is the overall deadline of when any follow-up amendments should be completed by",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa-ct-phase_2-3-approvals-2023-24.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["28 June 2024"]
},
{
  "query": "Who is responsible for checking the final information submitted for approval?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa-ct-phase_2-3-approvals-2023-24.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["Director of studies"]
},
{
  "query": "What are the type of revisions?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/quality-assurance-code-of-practice-approach-to-code-development.pdf"
  ],
  "expected_pages": [1,2],
  "expected_answers": ["Generic changes: Generic changes concern the updating of statements with new University terminology, changes in organisational structures or titles, or incorporating the consequences of other broad changes already approved by senior committees including Senate. Incremental changes: Incremental procedural and substantive changes are typically proposed on an ad hoc basis to address issues that have emerged, or to take forward enhancement opportunities which have been identified, through operation of the University’s routine quality management processes. These changes, alongside generic changes (see 2.1 above), make a significant contribution to ensuring that the statements remain relevant, accurate and current on an ongoing basis. Major reviews: Major reviews, which may take 1-2 years, will normally result in the major rewrite of a statement, or the creation of a new one, and require the approval of key principles by Senate. Such reviews will often be driven by university strategic priorities, and/or national developments. "]
},
{
  "query": "Who is required to take a compulsory unit?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [1,2],
  "expected_answers": ["Students working towards one of the generic awards"]
},
{
  "query": "What are the departments responsible for?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [5],
  "expected_answers": ["They are responsible for the learning, teaching and assessment on courses, units and programmes delivered to CPD students and the student learning experience of those students for whom they are identified as the “home” department. "]
},
{
  "query": "Who will periodically review the quality processes and principles underpinning the development?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["The University Learning, Teaching and Quality Committee"]
},
{
  "query": "What is the Doctoral responsible for??",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [3, 4, 5],
  "expected_answers": ["The Doctoral College is responsible for operational management of doctoral recruitment and admissions procedures; for providing administrative leadership including the management of processes relating to doctoral student progression, and quality and standards; for the identification, development and delivery of doctoral skills training, and for supporting and advising on all aspects relating to the support and enhancement of doctoral student provision;  The Doctoral College is responsible for organising the central induction and welcome for new doctoral students; additionally advising on the arrangement of Faculty/School/Departmental induction provision to ensure a consistent doctoral student experience across the institution. "]
},
{
  "query": "What is Mean Reciprocal Rank (MRR)?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [22],
  "expected_answers": ["mean reciprocal rank (MRR) measures how highly, on the average, the algorithm places the first relevant document in the ranked list that it returns"]
},
{
  "query": "What are the Lead supervisors responsible for?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["Lead Supervisors are responsible for supporting their doctoral students to progress satisfactorily and in accordance with the University’s Regulations and Quality Assurance expectations. Appendix 1 summarises the main responsibilities of the lead supervisor and supervisory team."]
},
{
  "query": "What are Doctoral students who are studying within a Doctoral Training Entity required to do?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [5],
  "expected_answers": ["Doctoral students who are studying within a Doctoral Training Entity may be required to attend further induction events organised by their training centre, which will outline the aims and requirements of their specific training programme "]
},
{
  "query": "What are the values that precision can take?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [22],
  "expected_answers": ["Precision can take values in the range of 0 to 1"]
},
{
  "query": "Where can the Candidate form be acessed?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [6],
  "expected_answers": ["The candidature form is accessed via SAMIS"]
},
{
  "query": "What is often referred as gold standard?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [21],
  "expected_answers": ["The number of documents that are returned by the algorithm within the top k, that are also relevant according to the “correct” relevance mapping (often referred to as the gold standard)"]
},
{
  "query": "What are the two main forms of research degree provision collaborative activity ?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["The two forms are individual student-based, and programme-based. Student-based: students who are registered on an individual basis for a doctoral award at another university conduct research at Bath, and students registered on an individual basis for a doctoral degree awarded solely by the University of Bath conduct some or all of their research elsewhere; Programme-based: students who are registered on a doctoral programme run by the University of Bath and at least one other institution and where the award(s) is/are made either by one institution, or separately by more than one institution."]
},
{
  "query": "What is the role of the External Examiner?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [22],
  "expected_answers": ["The role is examine the candidate’s suitability for the award of the higher degree in question, enable the University to ensure that its degrees are comparable in standard with, those awarded by other universities in the United Kingdom in similar subjects, verify that the standards expected of successful candidates are appropriate for the evel of the award, monitor and report on the proceedings of the Board of Examiners and in particular on whether these ensure that candidates are treated fairly and consistently."]
},
{
  "query": "In Declaration of Interests, What is the concerned party responsible for?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [32],
  "expected_answers": ["the concerned party is responsible for informing the Director of Studies, Head of Department/School or Dean of the existence of that relationship without delay. "]
},
{
  "query": "In Declaration of Interests, What is the concerned party responsible for?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa7-research-degrees.pdf"
  ],
  "expected_pages": [32],
  "expected_answers": ["the concerned party is responsible for informing the Director of Studies, Head of Department/School or Dean of the existence of that relationship without delay. "]
},
{
  "query": "What do the values of cosine similarity range between?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["The values range between 0 and 1"]
},
{
  "query": "What are the two ways of performing Constituency Parsing?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week4_Part-1copy.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["There are two ways: Bottom-up parsing and Top-down parsing"]
},
{
  "query": "What do the values of cosine similarity range between?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["The values range between 0 and 1"]
},
{
  "query": "What is Lemmatization?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [10],
  "expected_answers": ["The base form in each of the cases described above is always the most basic form of the word – it is the starting point for any further changes and formations. It is also the form of the word that you would find in the dictionary if you were to search for the meaning of the word or its spelling. Such a basic, dictionary form is also called lemma and the automated process of finding the lemma for an input form of the word is called lemmatization: for example, if you lemmatize the word form “sung” you will get its lemma “sing”."]
},
{
  "query": "What is Stemming?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [11],
  "expected_answers": ["An alternative to using lemmatization, which tries to match word forms to predefined dictionary forms, is to use an algorithm which relies on a set of language-specific morphological rules to analyse words on the go. This alternative approach is called stemming."]
},
{
  "query": "What is Stemming?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week2copy.pdf"
  ],
  "expected_pages": [11],
  "expected_answers": ["An alternative to using lemmatization, which tries to match word forms to predefined dictionary forms, is to use an algorithm which relies on a set of language-specific morphological rules to analyse words on the go. This alternative approach is called stemming."]
},
{
  "query": "What are context-free grammers?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week4_Part-1copy.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["Context-free grammars define the rules by which words are put together to produce grammatically correct well-formed sentences of English, where constituents can be recursively analysed and split into further sub-constituents "]
},
{
  "query": "What is chunking?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week4_Part-1copy.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": [" One approach to partial parsing is chunking – the process of identifying and classifying non-overlapping segments of a sentence that constitute the basic non-recursive phrases corresponding to the major parts- of-speech (i.e., including NPs, VPs, PPs, and APs – adjective phrases)."]
},
{
  "query": "What is the name of the simplest activation function?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week7_Part-1copy.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["The name of the simplest activation function is RELU"]
},
{
  "query": "What are the problems asscociated with the sigmoid and tanh function?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week7_Part-1copy.pdf"
  ],
  "expected_pages": [4],
  "expected_answers": ["In the sigmoid or tanh functions, very high values of z result in values of y that are saturated, i.e., extremely close to 1, and have derivatives very close to 0. Zero derivatives cause problems for learning"]
},
{
  "query": "What do the students have to do when they return from placement?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa6-placement-learning.pdf"
  ],
  "expected_pages": [9],
  "expected_answers": ["On return from placement, students will be asked to complete a survey evaluating their experience on placement, covering the academic elements of the unit and the level of support they received from the University."]
},
{
  "query": "Who is responsible for giving detailed consideration to the proposed amendment?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa20-collaborative-provision-taught-and-doctoral.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["F/SLTQC or F/SDSC"]
},
{
  "query": "What is a way to represent an input for classification?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week7_Part-1copy.pdf"
  ],
  "expected_pages": [12],
  "expected_answers": ["One simple baseline is to apply some sort of pooling function to the embeddings of all the words in the input. For example, for a text with n input words/tokens w1,...,wn, we can turn the n embeddings e(w1),...,e(wn) (each of dimensionality d) into a single embedding also of dimensionality d by just summing the embeddings, or by taking their mean (summing and then dividing by n)"]
},
{
  "query": "What is a way to represent an input for classification?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week7_Part-1copy.pdf"
  ],
  "expected_pages": [12],
  "expected_answers": ["One simple baseline is to apply some sort of pooling function to the embeddings of all the words in the input. For example, for a text with n input words/tokens w1,...,wn, we can turn the n embeddings e(w1),...,e(wn) (each of dimensionality d) into a single embedding also of dimensionality d by just summing the embeddings, or by taking their mean (summing and then dividing by n)"]
},
{
  "query": "What happens if a student fails the placement component?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa6-placement-learning.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["If a student fails the placement component of their course, on their return to full-time education within the University they will normally revert to studying on a related degree without placement, where this option exists."]
},
{
  "query": "What is the value of the Issue version written in this document?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa6-placement-learning.pdf"
  ],
  "expected_pages": [10],
  "expected_answers": ["The value of the Issue Version is 5.17"]
},
{
  "query": "What will be provided to distance learning students for introducing them?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa38-induction-of-students.pdf"
  ],
  "expected_pages": [8],
  "expected_answers": ["A suitable induction package will be provided to distance learning students introducing them to their course of study and the range of services available at the University."]
},
{
  "query": "What happens prior students going on placement?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa6-placement-learning.pdf"
  ],
  "expected_pages": [6],
  "expected_answers": ["Prior to going out on placement, all students will be fully briefed on their rights and responsibilities."]
},
{
  "query": "How do we estimate these bigram or n-gram probabilities?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week8_Part-1copy.pdf"
  ],
  "expected_pages": [4],
  "expected_answers": ["An intuitive way to estimate probabilities is called maximum likelihood estimation or MLE. We get the MLE estimate for the parameters of an n-gram model by getting counts from a corpus, and normalizing the counts so that they lie between 0 and 1."]
},
{
  "query": "Why is it important not to let the test sentences into the training set?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/Handout_Week8_Part-1copy.pdf"
  ],
  "expected_pages": [7],
  "expected_answers": ["Suppose we are trying to compute the probability of a particular “test” sentence. If our test sentence is part of the training corpus, we will mistakenly assign it an artificially high probability when it occurs in the test set. We call this situation training on the test set. Training on the test set introduces a bias that makes the probabilities all look too high, and causes huge inaccuracies in perplexity, the probability-based metric"]
},
{
  "query": "What are named awards?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["named awards (which would be entitled for example MSc Geographical Studies) are programmes normally developed as CPD for a specific sector and taken on a flexible unit-by-unit basis"]
},
{
  "query": "All students are allocated which depertment?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa56-continuing-professional-development.pdf"
  ],
  "expected_pages": [5],
  "expected_answers": ["All students are allocated a “home” department. "]
},
{
  "query": "What are the three distinct phases of the induction period?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa38-induction-of-students.pdf"
  ],
  "expected_pages": [2],
  "expected_answers": ["The three distinct phases are: Pre-arrival, (i.e., before students arrive on campus), Welcome Week, and Extended Induction. "]
},
{
  "query": "Where does the initial development of a new course of study takes place?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa3-approval-of-new-courses-of-study.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["The initial development of a new course of study takes place within a Department/ School/LPO or as a group development with others."]
},
{
  "query": "What is the Head of Department/School/LPO responsible for?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/qa3-approval-of-new-courses-of-study.pdf"
  ],
  "expected_pages": [3],
  "expected_answers": ["The Head of Department/School/LPO is responsible for incorporating an indication of planned new courses in departmental/School/partner college submissions during the annual planning round."]
},
{
  "query": "When do the amendments to the QA CoP take place?",
  "expected_documents": [
    "/Users/meheksawhney/Desktop/UI/attached_documents/quality-assurance-code-of-practice-approach-to-code-development.pdf"
  ],
  "expected_pages": [4],
  "expected_answers": ["Amendments to the QA CoP may take place with immediate effect, or from August in the following academic year."]
},
]



In [ ]:
## Computing the Precision and Recall, and F1-score for each query
def calculate_precision_recall(retrieved_docs, gold_standard_docs):
    # Convert lists to sets for easier calculation
    retrieved_set = set(retrieved_docs)
    gold_standard_set = set(gold_standard_docs)
    
    true_positives = len(retrieved_set.intersection(gold_standard_set))
    total_retrieved = len(retrieved_set)
    total_relevant = len(gold_standard_set)

    # Calculate precision and recall
    precision = true_positives / total_retrieved if total_retrieved > 0 else 0
    recall = true_positives / total_relevant if total_relevant > 0 else 0

    return precision, recall

results = []
for test in gold_standard:
    response, retrieved_documents, _ = query_check(test['query'], llama_model, documents, embeddingModel)
    precision, recall = calculate_precision_recall(retrieved_documents, test['expected_documents'])
    results.append((test['query'], precision, recall))

# Calculate F1-score for each result
f1_scores = [(2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0 for _, precision, recall in results]

# Calculate average precision, recall, and F1-score
average_precision = sum(precision for _, precision, _ in results) / len(results)
average_recall = sum(recall for _, _, recall in results) / len(results)
average_f1_score = sum(f1_scores) / len(f1_scores) if len(f1_scores) > 0 else 0

# Output averages
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1-score: {average_f1_score:.2f}")

import matplotlib.pyplot as plt
import numpy as np

# Graph showing precision and recall for each query 
# # queries = [result[0] for result in results]
# queries = ['Query {}'.format(i+1) for i in range(len(results))] 
# precisions = [result[1] for result in results]
# recalls = [result[2] for result in results]
# # f1_scores = [result[3] for result in results]

# # Create the bar chart with an increased figure size
# fig, ax = plt.subplots(figsize=(20, 10))  # Adjust the figure size as needed
# index = np.arange(len(queries))
# bar_width = 0.35

# ax.bar(index, precisions, bar_width, label='Precision')
# ax.bar(index + bar_width, recalls, bar_width, label='Recall')
# # ax.bar(index + 2 * bar_width, f1_scores, bar_width, label='F1 Score')

# ax.set_xlabel('Query', fontsize=14)
# ax.set_ylabel('Scores', fontsize=14)
# ax.set_title('Precision, Recall, and F1 Scores for Each Query', fontsize=18)
# ax.set_xticks(index + bar_width / 2)
# ax.set_xticklabels(queries, rotation=90)  
# ax.legend()

# plt.tight_layout()  
# plt.show()


# plt.show()

In [1]:
## Computing the BERTscore similarity
from evaluate import load

def evaluate_response(gold_standard, llama_model, documents, embeddingModel):
    rouge = load('rouge')
    bertscore = load('bertscore')
    results = []

    for item in gold_standard:
        query_text = item["query"]
        expected_answer = item["expected_answers"][0]

        # Call the query function
        result = query(query_text, llama_model, documents, embeddingModel)
        generated_response = result['response']['result']

        # Calculate BERTScore
        bertscore_result = bertscore.compute(predictions=[generated_response], references=[expected_answer], lang="en")
        
        results.append({
            "query": query_text,
            "expected_answer": expected_answer,
            "generated_response": generated_response,
            "bert_score": bertscore_result
        })

    return results

evaluation_results = evaluate_response(gold_standard, llama_model, documents, embeddingModel)
for result in evaluation_results:
    print(result)


In [ ]:
import numpy as np
from evaluate import load

def evaluate_response(gold_standard, llama_model, documents, embeddingModel):
    rouge = load('rouge')
    bertscore = load('bertscore')
    
    total_rouge1_f1, total_rouge2_f1, total_rougeL_f1 = 0, 0, 0
    total_bert_precision, total_bert_recall, total_bert_f1 = 0, 0, 0
    num_entries = len(gold_standard)

    for item in gold_standard:
        query_text = item["query"]
        expected_answer = item["expected_answers"][0]

        # Call the query function
        result = query(query_text, llama_model, documents, embeddingModel)
        generated_response = result['response']['result']

        # Calculate BERTScore
        bertscore_result = bertscore.compute(predictions=[generated_response], references=[expected_answer], lang="en")
        
        # Update BERTScore accumulators
        total_bert_precision += np.mean(bertscore_result["precision"])
        total_bert_recall += np.mean(bertscore_result["recall"])
        total_bert_f1 += np.mean(bertscore_result["f1"])

    # Calculate and return average scores only
    average_scores = {
        "average_bert_precision": total_bert_precision / num_entries,
        "average_bert_recall": total_bert_recall / num_entries,
        "average_bert_f1": total_bert_f1 / num_entries
    }

    return average_scores

average_scores = evaluate_response(gold_standard, llama_model, documents, embeddingModel)
print("Average Scores:", average_scores)



In [ ]:
# from rouge import Rouge
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.tokenize import sent_tokenize

# def evaluate_response(gold_standard, llama_model, documents, embeddingModel):
#     rouge = Rouge()
#     results = []

#     for item in gold_standard:
#         query_text = item["query"]
#         expected_answer = item["expected_answers"][0]  # Assuming there is only one expected answer per query
        
#         # Call the query function
#         result = query(query_text, llama_model, documents, embeddingModel)
#         generated_response = result['response']['result']  
#         # Calculate ROUGE and BLEU scores
#         rouge_scores = rouge.get_scores([generated_response], [expected_answer])
#         reference_sents = [sent_tokenize(expected_answer)]
#         generated_sents = sent_tokenize(generated_response)
#         bleu_score = sentence_bleu(reference_sents, generated_sents, weights=(0.25, 0.25, 0.25, 0.25))

#         results.append({
#             "query": query_text,
#             "expected_answer": expected_answer,
#             "generated_response": generated_response,
#             "rouge_scores": rouge_scores[0],  # Access the first element as get_scores returns a list of score dictionaries
#             "bleu_score": bleu_score
#         })

#     return results

# # Assuming 'llama_model', 'documents', and 'embeddingModel' are correctly defined and instantiated
# # Call the evaluation function
# evaluation_results = evaluate_response(gold_standard, llama_model, documents, embeddingModel)
# for result in evaluation_results:
#     print(result)


In [ ]:
# Assuming `llama_model`, `documents`, `embeddingModel` are defined and initialized appropriately
# results = []
# for test in gold_standard:
#     response, retrieved_documents, _ = query_check(test['query'], llama_model, documents, embeddingModel)
#     precision, recall = calculate_precision_recall(retrieved_documents, test['expected_documents'])
#     results.append((test['query'], precision, recall))

# # Output results
# for result in results:
#     print(f"Query: {result[0]}")
#     print(f"Precision: {result[1]:.2f}")
#     print(f"Recall: {result[2]:.2f}")
#     print("-------")

In [ ]:
# def evaluate_one_query(system_output, gold_standard):
#     predicted_docs = set(system_output[1])  # Assuming this is where document names are stored
#     true_docs = set(gold_standard['expected_documents'])

#     tp_docs = len(predicted_docs.intersection(true_docs))
#     fp_docs = len(predicted_docs) - tp_docs
#     fn_docs = len(true_docs) - tp_docs

#     precision_docs = (tp_docs / (tp_docs + fp_docs)) if (tp_docs + fp_docs) > 0 else 0
#     recall_docs = (tp_docs / (tp_docs + fn_docs)) if (tp_docs + fn_docs) > 0 else 0
#     f1_docs = 2 * (precision_docs * recall_docs) / (precision_docs + recall_docs) if (precision_docs + recall_docs) > 0 else 0

#     return precision_docs, recall_docs, f1_docs

In [ ]:
# results = []
# for test in gold_standard:
#     output = query_check(test['query'], llama_model, documents, embeddingModel)
#     precision, recall, f1 = evaluate_one_query(output, test)
#     results.append((precision, recall, f1))

# # Print the average of the metrics across all queries
# average_precision = sum(r[0] for r in results) / len(results)
# average_recall = sum(r[1] for r in results) / len(results)
# average_f1 = sum(r[2] for r in results) / len(results)

# print(f"Average Precision for documents: {average_precision:.2f}")
# print(f"Average Recall for documents: {average_recall:.2f}")
# print(f"Average F1 Score for documents: {average_f1:.2f}")

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score

# # Define a function to evaluate one query-response pair
# def evaluate_query_response(response, gold_standard):
#     predicted_docs = set(response['document_names'])
#     predicted_pages = set(response['pages'])
    
#     true_docs = set(gold_standard['expected_documents'])
#     true_pages = set(gold_standard['expected_pages'])

#     # Calculate accuracy metrics
#     precision_docs = len(predicted_docs.intersection(true_docs)) / len(predicted_docs)
#     recall_docs = len(predicted_docs.intersection(true_docs)) / len(true_docs)
#     f1_docs = 2 * (precision_docs * recall_docs) / (precision_docs + recall_docs) if (precision_docs + recall_docs) > 0 else 0

#     precision_pages = len(predicted_pages.intersection(true_pages)) / len(predicted_pages)
#     recall_pages = len(predicted_pages.intersection(true_pages)) / len(true_pages)
#     f1_pages = 2 * (precision_pages * recall_pages) / (precision_pages + recall_pages) if (precision_pages + recall_pages) > 0 else 0

#     return {
#         'precision_docs': precision_docs,
#         'recall_docs': recall_docs,
#         'f1_docs': f1_docs,
#         'precision_pages': precision_pages,
#         'recall_pages': recall_pages,
#         'f1_pages': f1_pages
#     }

# # Evaluate all query-response pairs
# results = []
# for gold_query_response in gold_standard:
#     query_text = gold_query_response['query']
#     gold_response = {
#         'document_names': gold_query_response['expected_documents'],
#         'pages': gold_query_response['expected_pages']
#     }
#     system_response = query(query_text, llama_model, documents, embeddingModel) 
#     result = evaluate_query_response(system_response, gold_query_response)
#     results.append(result)

# # Calculate average metrics across all query-response pairs
# avg_precision_docs = sum(result['precision_docs'] for result in results) / len(results)
# avg_recall_docs = sum(result['recall_docs'] for result in results) / len(results)
# avg_f1_docs = sum(result['f1_docs'] for result in results) / len(results)

# avg_precision_pages = sum(result['precision_pages'] for result in results) / len(results)
# avg_recall_pages = sum(result['recall_pages'] for result in results) / len(results)
# avg_f1_pages = sum(result['f1_pages'] for result in results) / len(results)

# print("Average Precision for Documents:", avg_precision_docs)
# print("Average Recall for Documents:", avg_recall_docs)
# print("Average F1 Score for Documents:", avg_f1_docs)
# print("Average Precision for Pages:", avg_precision_pages)
# print("Average Recall for Pages:", avg_recall_pages)
# print("Average F1 Score for Pages:", avg_f1_pages)


In [ ]:
# import matplotlib.pyplot as plt

# # Document counts and corresponding response times
# textual_docs = [10, 20, 30, 40, 100]
# textual_times = [76, 101, 133, 158, 303]  # Converted to seconds

# image_docs = [5]
# image_times = [365]  

# # Create the plot
# plt.figure(figsize=(5, 4))

# # Plotting textual documents
# plt.plot(textual_docs, textual_times, label='Textual Documents', marker='o')

# # Plotting image document
# plt.scatter(image_docs, image_times, color='red', label='Image Document', s=100)  # Use scatter for single points

# # Adding titles and labels
# plt.title('Response Time VS Number of Documents')
# plt.xlabel('Number of Documents')
# plt.ylabel('Response Time (seconds)')

# # Setting x-axis ticks manually
# plt.xticks([5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

# # Adding a legend
# plt.legend()

# # Show the plot
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# # Data for plotting
# tasks = ['Accuracy', 'Location', 'Ease', 'Reusability Rate']
# scores = [8.5, 9.7, 9.0, 9.0]

# # Creating the bar chart
# plt.figure(figsize=(5, 4))
# plt.bar(tasks, scores, color=['blue', 'green', 'red', 'purple'])
# plt.xlabel('Tasks')
# plt.ylabel('Average Rating')
# plt.title('Quantitative Results from Usability Testing')
# plt.ylim(8, 10)  # Setting the y-axis limit to better highlight differences
# plt.axhline(y=9, color='gray', linestyle='--')  # Adding a horizontal line at the average score level
# # plt.xticks(rotation=90)  # Rotating x-axis labels to 90 degrees
# plt.show()

